In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

driver = webdriver.Chrome()

driver.get("https://www.daraz.pk/smartphones")
import requests
daraz_page = 1
data = []
reviews_data = []
specs_data = []
UID = 0

In [38]:

# Loop over first 5 pages
for i in range(1, 6):
    bottom = 315
    top = 0
    
    mobiles = driver.find_elements(By.CSS_SELECTOR, '.box--ujueT .gridItem--Yd0sa')
# Iterate over all the mobiles in the main container
    for mob_index, mobile in enumerate(mobiles):
        UID += 1
        try:
            title_element = mobile.find_element(By.CSS_SELECTOR, '#id-a-link')
            url = title_element.get_attribute('href')
            
            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[1])
            driver.get(url)
            time.sleep(1)     
            
#           Price
            try:
                price = driver.find_element(By.XPATH,
                                            '/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[7]/div/div/span').text
                price = price[4:]
                price = price.replace(',', '')
            except NoSuchElementException:
                price = "0"

            time.sleep(1)
            driver.execute_script("window.scrollTo(0, 500)")
            time.sleep(1)
            
            
#            specs
            driver.execute_script("window.scrollTo(1000, 2000)")
            specs_dict = {'UID': UID}  # Initialize outside the loop

            up = 1000
            down = 1500
            try:
                while True:
                    specs_container = driver.find_elements(By.CSS_SELECTOR, '.key-li')

                    view_more_button = None
                    try:
                        driver.execute_script(f"window.scrollTo({up}, {down})")
                        view_more_button = driver.find_element(By.CSS_SELECTOR, '.expand-button button.pdp-view-more-btn')
                        view_more_button.click()
                        up = down
                        time.sleep(1)
                        down += 500
                    except:
                        pass

                    if not specs_container and not view_more_button:
                        break

                    for spec in specs_container:
                        title_spec = spec.find_element(By.CLASS_NAME, 'key-title').text
                        detail_spec = spec.find_element(By.CSS_SELECTOR, '.html-content.key-value').text
                        specs_dict[title_spec] = detail_spec  # Append to the existing dictionary

                    if specs_container:
                        break

            except Exception as e:
                print(f"An error occurred: {e}")

            # appending specs_data list
            specs_data.append(specs_dict)
            
            try:
                           
                driver.execute_script(f"window.scrollTo({up}, {down})")
                questions_text = driver.find_element(By.XPATH, '//*[@id="module_product_qna"]/div/div[1]/div').text

                match = re.search(r'\((\d+)\)', questions_text)

                if match:
                    # numbers from the matched group
                    question_number = match.group(1)

            except Exception as e:
                print(f"An error occurred: {e}")
            
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            
            
#           rating 
            try:
                rating_element = driver.find_element(By.CSS_SELECTOR, '.review-info-rate .score')
                rating = rating_element.text
            except NoSuchElementException:
                rating = 0.0

            driver.execute_script(f"window.scrollTo(500, 1000)")
            RID = 0
            
#            reviews
            for page in range(0, 2):
                review = []

                try:
                    reviews_container = driver.find_element(By.CSS_SELECTOR, '#module_product_review')
                except NoSuchElementException:
                    print('No Reviews')
                    break

                reviews = reviews_container.find_elements(By.CLASS_NAME, 'review-item')

                for i in range(len(reviews)):
                    if reviews[i].find_element(By.CSS_SELECTOR, '.review-content-sl').text != '':
                        review_name = reviews[i].find_element(By.CLASS_NAME, 'user').text
                        review_content = reviews[i].find_element(By.CSS_SELECTOR, '.review-content-sl').text
                        RID += 1

                        # Counting the star images
                        star_images = reviews[i].find_elements(By.XPATH, './/img[contains(@src, "//laz-img-cdn.alicdn.com/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png")]')
                        Review_Rating = len(star_images)

                        review.append({'UID': UID, 'Review_ID': RID, 'Name': review_name, 'Rating': Review_Rating, 'Text': review_content})

                reviews_data.append(review)
                
                #Title 
                #Brand
                try:
                    title = driver.find_element(By.XPATH,
                                                '/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[3]/div/div/span').text
                except NoSuchElementException:
                    title = "N/A"

                brand = title.split(' ')[0]

#                 try:
#                     if page != 2:
#                         nxt_button =  driver.find_element(By.XPATH, '/html/body/div[3]/div/div[4]/div[1]/div/div[1]/div[3]/div[2]/div/ul/li[9]')
#                         driver.execute_script("arguments[0].click();", nxt_button)
#                         time.sleep(2)

#                 except NoSuchElementException:
#                     break

        
            data.append({'UID': UID, 'Name': title, 'Brand': brand, 'Price': price, 'Rating': rating, 'Questions Count' : question_number ,'URL' : url})
            
        except Exception as e:
            print(f"Error processing a mobile element: {e}")

        if mob_index % 3 == 0:
            driver.execute_script(f"window.scrollTo({top},{bottom})")
            top = bottom
            bottom += 315
            time.sleep(1)
  


    while daraz_page < 6:
        daraz_page += 1 
    next_page =   driver.find_element(By.XPATH, '/html/body/div[3]/div/div[4]/div[1]/div/div[1]/div[3]/div[2]/div/ul/li[9]')  
    next_page.click()
    time.sleep(1)
        

No Reviews
An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="module_product_qna"]/div/div[1]/div"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF708DF82B2+55298]
	(No symbol) [0x00007FF708D65E02]
	(No symbol) [0x00007FF708C205AB]
	(No symbol) [0x00007FF708C6175C]
	(No symbol) [0x00007FF708C618DC]
	(No symbol) [0x00007FF708C9CBC7]
	(No symbol) [0x00007FF708C820EF]
	(No symbol) [0x00007FF708C9AAA4]
	(No symbol) [0x00007FF708C81E83]
	(No symbol) [0x00007FF708C5670A]
	(No symbol) [0x00007FF708C57964]
	GetHandleVerifier [0x00007FF709170AAB+3694587]
	GetHandleVerifier [0x00007FF7091C728E+4048862]
	GetHandleVerifier [0x00007FF7091BF173+4015811]
	GetHandleVerifier [0x00007FF708E947D6+695590]
	(No symbol) [0x00007FF708D70CE8]
	(No symbol) [0x00007FF708D6CF34

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div/div[4]/div[1]/div/div[1]/div[3]/div[2]/div/ul/li[9]"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF708DF82B2+55298]
	(No symbol) [0x00007FF708D65E02]
	(No symbol) [0x00007FF708C205AB]
	(No symbol) [0x00007FF708C6175C]
	(No symbol) [0x00007FF708C618DC]
	(No symbol) [0x00007FF708C9CBC7]
	(No symbol) [0x00007FF708C820EF]
	(No symbol) [0x00007FF708C9AAA4]
	(No symbol) [0x00007FF708C81E83]
	(No symbol) [0x00007FF708C5670A]
	(No symbol) [0x00007FF708C57964]
	GetHandleVerifier [0x00007FF709170AAB+3694587]
	GetHandleVerifier [0x00007FF7091C728E+4048862]
	GetHandleVerifier [0x00007FF7091BF173+4015811]
	GetHandleVerifier [0x00007FF708E947D6+695590]
	(No symbol) [0x00007FF708D70CE8]
	(No symb

In [39]:
import pandas as pd
df=pd.DataFrame(data)
df.to_csv('proj.csv',index=False)

In [40]:
df1=pd.DataFrame(reviews_data)
df2=pd.DataFrame(specs_data)
main_df=pd.concat([df,df1,df2],axis=1)






In [42]:
df=pd.DataFrame(main_df)
df.to_csv('proj2.csv',index=False)

In [35]:
reviews_data


[[{'UID': 1,
   'Review_ID': 1,
   'Name': 'Muhammad B.',
   'Rating': 5,
   'Text': "Unbelievable! phone is 100 percent original and value for money, don't know how seller managed to offer the phone in such a cheap price, moreover its officially PTA approved. seller deserves 10 stars"},
  {'UID': 1,
   'Review_ID': 2,
   'Name': 'Musadique S.',
   'Rating': 5,
   'Text': 'Genuine samsung mobile, well packed, pta approved. I like it. Very beautiful mobile and hope it will be long lasting experience. #11.11'},
  {'UID': 1,
   'Review_ID': 3,
   'Name': 'S***.',
   'Rating': 5,
   'Text': '#11.11 Mobile without mobile warranty card, charger but seems good, packing was not good delivery is pathatic'}],
 [{'UID': 2,
   'Review_ID': 1,
   'Name': 'Gohar I.',
   'Rating': 5,
   'Text': 'this is the initial review. the phone is the same as shown.good thing is that it is managed directly by Daraz. packaging was amazing. order came 1 day late only.'},
  {'UID': 2,
   'Review_ID': 2,
   'Name': 

In [34]:
specs_data


[{'UID': 1,
  'Brand': 'No Brand',
  'SKU': '436839264_PK-2101404094',
  'Protection': 'Not Specified',
  'Year': '2020',
  'Number Of Cameras': 'Not Specified'},
 {'UID': 2,
  'Brand': 'Xiaomi',
  'SKU': '429185340_PK-2040915791',
  'Protection': 'Not Specified',
  'Year': 'Not Specified',
  'Number Of Cameras': 'Not Specified'},
 {'UID': 3,
  'Brand': 'No Brand',
  'SKU': '436821010_PK-2101334479',
  'Protection': 'Not Specified',
  'Year': 'Not Specified',
  'Number Of Cameras': 'Not Specified'},
 {'UID': 4,
  'Brand': 'Xiaomi',
  'SKU': '430451931_PK-2051820850',
  'Protection': 'Not Specified',
  'Year': 'Not Specified',
  'Number Of Cameras': '2'}]